<a href="https://colab.research.google.com/github/WillN202/NLU_CW/blob/main/NLU_Task2_BERTEncodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLU Task 2


For this task I've implemented a GRU model using TODO word embeddings. TODO WRITE ME

## Imports and Setup

In [1]:
!pip install torchmetrics
!pip install ray[tune]
!pip install -U sentence-transformers
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 7.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchmetrics
import os.path
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from ray import tune, train
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch
import string
import re
import gensim.downloader
import tempfile
import random
TRAINING_DATASET_LOCATION = "/content/drive/MyDrive/train.csv"
DEV_DATASET_LOCATION = "/content/drive/MyDrive/dev.csv"
TEST_DATASET_LOCATION = "/content/drive/MyDrive/AV_trial.csv"
WORD2VEC_EMBEDDINGS = "/content/drive/MyDrive/word2vec_embeddings_tessty.model"
EPOCHS = 4

DEVICE = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(DEVICE)
# torch.set_default_device(DEVICE)

## Data Loading and Pre Processing

In [ ]:
def generic_preprocessor(sentence):
  sentence = sentence.lower()

  return sentence

class AVDataset(Dataset):
  def __init__(self, csv_file, pre_processor=None):
    self.samples = pd.read_csv(csv_file)
    self.pre_processor = pre_processor

  def __len__(self):
      return len(self.samples)

  def __getitem__(self, index):
    sample = self.samples.iloc[index]
    # sample[0] = re.sub(f"[{re.escape(string.punctuation)}]", "", sample[0])
    # sample[0] = re.sub(f"[{re.escape(string.punctuation)}]", "", sample[1])
    sample_text = f"{sample[0]} <sep> {sample[1]}"
    return (self.pre_processor(sample_text), sample[2])


In [ ]:
training_samples = AVDataset(TRAINING_DATASET_LOCATION, pre_processor=generic_preprocessor)
dev_samples = AVDataset(DEV_DATASET_LOCATION, pre_processor=generic_preprocessor)
test_samples = AVDataset(TEST_DATASET_LOCATION, pre_processor=generic_preprocessor)

## Word Embedding Generation

## Model Creation

To reduce code content, the GRU models are defined from a base class. New linear layers are added between each class of model since this could not be done using parameters alone

In [ ]:
class BaseGruRNN(torch.nn.Module):
  def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
    super(BaseGruRNN, self).__init__()
    self.vocab = vocab
    self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
    self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True, num_layers=rnn_layers, dropout=0.1, bidirectional=is_bidirectional)

  def forward(self, x, linear_layer):
    unk_embedding = self.vocab["<UNK>"]
    #x = [re.sub(f"[{re.escape(string.punctuation)}]", "", sentence) for sentence in x]
    x = [sentence.split() for sentence in x]
    x = [[self.vocab.get(word, unk_embedding) for word in sentence ] for sentence in x]
    # TODO -> instead of padding, use pack sequence instead. Note this may break the output from the lstm (woo)
    max_len = max([len(words) for words in x])
    x = [([self.vocab["<pad>"]] * (max_len -  len(words))) + words for words in x]
    input = torch.tensor(x)

    embeddings = self.get_embedding(input)
    GRU_int_results = self.GRU_Layer(embeddings)[1]
    GRU_values = GRU_int_results[-1] #Get last hidden state(s)
    result = linear_layer(GRU_values)
    return result

class OneLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      self.base = super(OneLinearLayerGruRNN, self).__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, output_size),
      )

    def forward(self, x):
      return super().forward(x, self.linear_layer)

class TwoLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      super().__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )

    def forward(self, x):
      return super().forward(x, self.linear_layer)

class ThreeLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      super(ThreeLinearLayerGruRNN, self).__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )

    def forward(self, x):
      return super().forward(x, self.linear_layer)

class FourLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      super(FourLinearLayerGruRNN, self).__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )


    def forward(self, x):
      return super().forward(x, self.linear_layer)


## Model Training

In [ ]:
def train_and_validate_loop(model, batch_size, learning_rate, epochs, tuning=False):
  device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "cpu"
)
  torch.set_default_device(device)
  loss_function = torch.nn.BCEWithLogitsLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  training_loader = DataLoader(training_samples, batch_size=batch_size, generator=torch.Generator(device=device))
  dev_loader = DataLoader(dev_samples, batch_size=batch_size, generator=torch.Generator(device=device))
  accuracy = torchmetrics.Accuracy(task="binary")
  f1_score = torchmetrics.F1Score(task="binary")
  mcc = torchmetrics.MatthewsCorrCoef(task="binary")
  torch.set_grad_enabled(True)

  if not tuning:
    print(f"Epochs: {epochs}")

  for epoch in range(0, epochs):
      model.train()
      running_loss = 0.0

      # Training
      for index, value in enumerate(training_loader):
          optimizer.zero_grad()
          data, labels = value
          labels = labels.reshape(-1,1)
          labels = labels.type(torch.FloatTensor)
          labels = labels.to(device)
          outputs = model(data)

          loss = loss_function(outputs, labels)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()

      # Validation to ensure the model is learning
      model.eval()
      torch.set_grad_enabled(False)
      running_loss = 0
      num_correct = 0

      pred = torch.Tensor().to(device)
      gold_standard = torch.Tensor().to(device)
      for index, value in enumerate(dev_loader):
          data, labels = value
          labels = labels.reshape(-1,1)
          labels = labels.type(torch.FloatTensor)
          labels = labels.to(device)

          outputs = model(data)
          loss = loss_function(outputs, labels)
          running_loss += loss

          normalised_outputs = torch.sigmoid(outputs)
          pred = torch.cat((pred, normalised_outputs))
          gold_standard = torch.cat((gold_standard, labels))

      torch.set_grad_enabled(True)
      batch_loss = running_loss / len(dev_loader)
      f1 = f1_score(pred, gold_standard)
      acc = accuracy(pred, gold_standard)
      m_score = mcc(pred, gold_standard)

      score = float(f1) + (2 * float(acc)) + (2.5 * float(m_score))
      with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
        checkpoint = None
        if (epoch + 1) % 3 == 0:
            # This saves the model to the trial directory
            torch.save(
                model.state_dict(),
                os.path.join(temp_checkpoint_dir, "model.pth")
            )
            checkpoint = train.Checkpoint.from_directory(temp_checkpoint_dir)

        if tuning:
          train.report({"score": score}, checkpoint=checkpoint)
        else:
          print(f"---------------------EPOCH {epoch+1} / {epochs}---------------------")
          print(f"Batch Loss {batch_loss}")
          print(f"Accuracy {accuracy(pred, gold_standard)}")


In [ ]:
# test_embeddings = gensim.downloader.load("word2vec-google-news-300")
# test_embeddings["<UNK>"] = np.random.rand(300)
# test_embeddings["<sep>"] = np.random.rand(300)
# test_embeddings["<pad>"] = np.random.rand(300)
# coded_embeddings = torch.FloatTensor(test_embeddings.vectors).to(DEVICE)
# vocab = test_embeddings.key_to_index
# model = FourLinearLayerGruRNN(300, 1, 300, coded_embeddings, vocab, 1, is_bidirectional=True).to(DEVICE)
# train_and_validate_loop(model,64, 0.000019238, 3)

## Hyperparameter Selection

In [ ]:
def tune_training_step(config, embedding, network):
  device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "cpu"
)

  chosen_network = network[config["linear_layers"]]
  embedding = embedding[config["embedding_size"]]
  coded_embeddings = torch.FloatTensor(np.array(embedding.wv.vectors)).to(device)
  vocab = embedding.wv.key_to_index
  generated_model = chosen_network(config["embedding_size"], 1, config["embedding_size"], coded_embeddings, vocab, config["rnn_layers"], config["is_bidirectional"]).to(device)
  train_and_validate_loop(generated_model, config["batch_size"], config["lr"], config["epochs"], True)


config = {
  "lr": tune.loguniform(1e-6, 1),
  "batch_size": tune.choice([8, 16, 32, 64, 128]),
  "is_bidirectional": tune.choice([True, False]),
  "epochs": tune.choice(list(range(3,15))),
  "linear_layers": tune.choice(list(range(1,5))),
  "rnn_layers": tune.choice(list(range(1,5))),
  "embedding_size": tune.choice([64,128,256,512, 1024]),
  "hidden_size": tune.choice(list(range(32, 513, 32)))
}


embedding = {
    64: embeddings_64,
    128: embeddings_128,
    256: embeddings_256,
    512: embeddings_512,
    1024: embeddings_1024
}

network = {
    1: OneLinearLayerGruRNN,
    2: TwoLinearLayerGruRNN,
    3: ThreeLinearLayerGruRNN,
    4: FourLinearLayerGruRNN
}

asha_scheduler = ASHAScheduler(
        metric="score",
        mode="max",
        max_t=14,
        grace_period=6,
        reduction_factor=2
    )

chosen_search_alg = HyperOptSearch(metric="score", mode="max")

training_wrapper = tune.with_resources(tune.with_parameters(tune_training_step, embedding=embedding, network=network), {"CPU": 1.6, "GPU": 0.25})
tuner = tune.Tuner(
    training_wrapper,
    tune_config=tune.TuneConfig(
        num_samples=40,
        search_alg=chosen_search_alg,
        scheduler=asha_scheduler,
    ),
    param_space=config
)

result = tuner.fit()


2024-04-23 22:39:44,853	INFO tune.py:633 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+---------------------------------------------------------------------------+
| Configuration for experiment     tune_training_step_2024-04-23_22-39-44   |
+---------------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                          |
| Scheduler                        AsyncHyperBandScheduler                  |
| Number of trials                 40                                       |
+---------------------------------------------------------------------------+

View detailed results here: /root/ray_results/tune_training_step_2024-04-23_22-39-44
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-23_22-38-25_456352_387/artifacts/2024-04-23_22-39-45/tune_training_step_2024-04-23_22-39-44/driver_artifacts`
(raylet) Warning: The actor ImplicitFunc is very large (38 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip

(tune_training_step pid=24568) /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
(tune_training_step pid=24568)   warnings.warn("dropout option adds dropout after all but last "



Trial tune_training_step_5c2f19b7 started with configuration:
+----------------------------------------------------+
| Trial tune_training_step_5c2f19b7 config           |
+----------------------------------------------------+
| batch_size                                      64 |
| embedding_size                                 256 |
| epochs                                           7 |
| hidden_size                                     32 |
| is_bidirectional                                 0 |
| linear_layers                                    3 |
| lr                                           9e-05 |
| rnn_layers                                       4 |
+----------------------------------------------------+

Trial tune_training_step_8ccad67c started with configuration:
+--------------------------------------------------+
| Trial tune_training_step_8ccad67c config         |
+--------------------------------------------------+
| batch_size                                     8 |
| 

(tune_training_step pid=24568) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/tune_training_step_2024-04-23_22-39-44/tune_training_step_ebeea738_1_batch_size=32,embedding_size=64,epochs=11,hidden_size=384,is_bidirectional=True,linear_layers=3,lr=0_2024-04-23_22-39-45/checkpoint_000000)


...


Trial tune_training_step_dbe0c4a8 started with configuration:
+------------------------------------------------------+
| Trial tune_training_step_dbe0c4a8 config             |
+------------------------------------------------------+
| batch_size                                        16 |
| embedding_size                                   512 |
| epochs                                             7 |
| hidden_size                                      128 |
| is_bidirectional                                   0 |
| linear_layers                                      2 |
| lr                                           0.89516 |
| rnn_layers                                         4 |
+------------------------------------------------------+


(tune_training_step pid=26886) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/tune_training_step_2024-04-23_22-39-44/tune_training_step_ac9ccd2e_7_batch_size=32,embedding_size=512,epochs=14,hidden_size=352,is_bidirectional=True,linear_layers=3,lr=_2024-04-23_22-45-48/checkpoint_000000)



Trial status: 6 TERMINATED | 4 RUNNING | 1 PENDING
Current time: 2024-04-23 22:55:47. Total running time: 16min 2s
Logical resource usage: 6.4/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                    status                lr     batch_size   is_bidirectional       epochs     linear_layers     rnn_layers     embedding_size     hidden_size     iter     total time (s)      score |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| tune_training_step_ac9ccd2e   RUNNING      0.0475083               32   True                       14                 3              2                512            

(tune_training_step pid=27935) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/tune_training_step_2024-04-23_22-39-44/tune_training_step_33d644c6_8_batch_size=8,embedding_size=64,epochs=5,hidden_size=64,is_bidirectional=False,linear_layers=2,lr=0.1_2024-04-23_22-47-37/checkpoint_000000)


Trial status: 6 TERMINATED | 4 RUNNING | 1 PENDING
Current time: 2024-04-23 22:57:17. Total running time: 17min 32s
Logical resource usage: 6.4/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                    status                lr     batch_size   is_bidirectional       epochs     linear_layers     rnn_layers     embedding_size     hidden_size     iter     total time (s)      score |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| tune_training_step_ac9ccd2e   RUNNING      0.0475083               32   True                       14                 3              2                512            

## Model Evaluation

In [ ]:
best_result = result.get_best_result("score", mode="max")
with best_result.checkpoint.as_directory() as checkpoint_dir:
    state_dict = torch.load(os.path.join(checkpoint_dir, "model.pth"))

print(best_result)

Result(
  metrics={'score': 0.6682943105697632},
  path='/root/ray_results/tune_training_step_2024-04-23_20-01-30/tune_training_step_d96db9c6_5_batch_size=64,embedding_size=512,epochs=3,is_bidirectional=False,linear_layers=4,lr=0.1260,rnn_layer_2024-04-23_20-01-53',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/root/ray_results/tune_training_step_2024-04-23_20-01-30/tune_training_step_d96db9c6_5_batch_size=64,embedding_size=512,epochs=3,is_bidirectional=False,linear_layers=4,lr=0.1260,rnn_layer_2024-04-23_20-01-53/checkpoint_000000)
)


In [ ]:
chosen_embedding = embeddings_512
coded_embeddings = torch.FloatTensor(chosen_embedding.wv.vectors).to(DEVICE)
vocab = chosen_embedding.wv.key_to_index
model = FourLinearLayerGruRNN(512, 1, 512, coded_embeddings, vocab, 1, is_bidirectional=False).to(DEVICE)
model.load_state_dict(state_dict)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


<All keys matched successfully>

In [ ]:
model.eval()
torch.set_default_device(DEVICE)
torch.set_grad_enabled(False)
num_correct = 0
loader = DataLoader(test_samples, batch_size=64, generator=torch.Generator(device=DEVICE))

answers = torch.Tensor().to(DEVICE)
gold_standard = torch.Tensor().to(DEVICE)

for index, value in enumerate(loader):
    data, labels = value
    labels = labels.reshape(-1,1)
    labels = labels.type(torch.FloatTensor)
    labels = labels.to(DEVICE)

    outputs = model(data)

    answers = torch.cat((answers, outputs))
    gold_standard = torch.cat((gold_standard, labels))


torch.set_grad_enabled(True)
answers = torch.sigmoid(answers)
accuracy = torchmetrics.Accuracy(task="binary")
mcc = torchmetrics.MatthewsCorrCoef(task="binary")
f1_score = torchmetrics.F1Score(task="binary")
print(f"Accuracy {accuracy(answers, gold_standard)}")
print(f"MCC {mcc(answers, gold_standard)}")
print(f"F1 {f1_score(answers, gold_standard)}")


Accuracy 0.5
MCC 0.0
F1 0.6666666865348816
